# PROJET 3 - Exportation vers SQLite et résultats des requêtes SQL

Pour préparer les données afin de procéder à l'exportation vers SQL, nous allons requêter grâce à Pandas. Nous avons créé une connexion grâce au module sqlite3 qui nous permet d'afficher le résultat des requêtes SQL.

## Importation des modules

In [1]:
import pandas as pd
import sqlite3

## Chargement des données

In [2]:
%pwd

'/Users/anissa/PROJET 3'

In [3]:
%cd /Users/anissa/FAO_2013_fr

/Users/anissa/FAO_2013_fr


In [4]:
population = pd.read_csv("fr_population.csv")
vegetaux = pd.read_csv("fr_vegetaux.csv")
animaux = pd.read_csv("fr_animaux.csv")
sous_alimentation = pd.read_csv("fr_sousalimentation.csv")

## Exportation vers la base de données

Pour procéder à l'exécution de nos requêtes SQL, il nous faut créer les dataframes.

### Question n°15 - Export df population

In [5]:
# On travaille sur le df population afin de l'exporter

# On sélectionne le df en supprimant la ligne correspondant à la Chine
pop = population.loc[population["Symbole"]!="A"]
# On sélectionne les colonnes dont on a besoin
col_list = ["Code zone","Zone","Année"]
population = pop[col_list]
# On les renomme
population = population.rename(columns={"Code zone":"code_pays","Zone":"pays",
                                                                        "Année":"annee"})
# On ajoute la colonne population multipliée par 1000 (unité)
population["population"] = pop["Valeur"]*1000
population.head()

,code_pays,pays,annee,population
0,2,Afghanistan,2013,30552000
1,202,Afrique du Sud,2013,52776000
2,3,Albanie,2013,3173000
3,4,Algérie,2013,39208000
4,79,Allemagne,2013,82727000


*clé primaire (code_pays,annee)*

In [6]:
# On procède à l'exportation SQL
population.to_csv("/Users/anissa/pop.csv", index = False)

### Question n°16 - Export df dispo_alim

In [7]:
# On travaille sur le df dispo_alim afin de l'exporter 

# On procède à la concaténation des df animaux et végétaux
# On ajoute d'abord une colonne origine dans chaque df
vegetaux["Origine"]="Végétale"
animaux["Origine"]="Animale"
# On lance la concaténation
anim_veg = pd.concat([vegetaux,animaux])
# On exclut les lignes "Chine" (redondance)
anim_veg = anim_veg.loc[anim_veg["Code zone"]!=351]
# On crée un tableau croisé dynamique (table pivot)
# On commence par supprimer la colonne Code Élément
anim_veg_new = anim_veg.drop(columns=["Code Élément"])
# On crée le table pivot
anim_veg_pivot = pd.pivot_table(anim_veg_new, index=["Code zone", "Zone","Code Produit", 
                                                     "Produit","Origine","Année",
                                                     "Code année"],
                                                   columns="Élément", values="Valeur", 
                                                                aggfunc="sum")
# On réinitialise les indexs
bilan_alim= anim_veg_pivot.reset_index()
# On sélectionne les colonnes dont on a besoin
col_list = ["Code zone","Zone","Année","Produit","Code Produit","Origine",
            "Disponibilité alimentaire en quantité (kg/personne/an)",
            "Disponibilité alimentaire (Kcal/personne/jour)",
            "Disponibilité de protéines en quantité (g/personne/jour)",
            "Disponibilité de matière grasse en quantité (g/personne/jour)"]
dispo_alim = bilan_alim[col_list]
# On les renomme
dispo_alim = dispo_alim.rename(columns={"Code zone":"code_pays","Zone":"pays",
                                        "Année":"annee","Produit":"produit",
                                        "Code Produit":"code_produit","Origine":"origin",
            "Disponibilité alimentaire en quantité (kg/personne/an)":"dispo_alim_tonnes",
                "Disponibilité alimentaire (Kcal/personne/jour)":"dispo_alim_kcal_p_j",
                "Disponibilité de protéines en quantité (g/personne/jour)":"dispo_prot",
        "Disponibilité de matière grasse en quantité (g/personne/jour)":"dispo_mat_gr"})
dispo_alim.head()

Élément,code_pays,pays,annee,produit,code_produit,origin,dispo_alim_tonnes,dispo_alim_kcal_p_j,dispo_prot,dispo_mat_gr
0,1,Arménie,2013,Blé,2511,Végétale,130.60,1024.0,30.52,3.60
1,1,Arménie,2013,Orge,2513,Végétale,0.00,0.0,0.00,0.00
2,1,Arménie,2013,Maïs,2514,Végétale,0.03,0.0,0.01,NaN
3,1,Arménie,2013,Seigle,2515,Végétale,0.12,1.0,0.02,0.00
4,1,Arménie,2013,Avoine,2516,Végétale,0.37,2.0,0.09,0.03


*clé primaire (code_pays, code_produit, annee)*

In [8]:
# On procède à l'exportation SQL
dispo_alim.to_csv("/Users/anissa/dispo_alim.csv", index = False)

### Question n°17 - Export df equilibre_prod

In [9]:
# On sélectionne les colonnes dont on a besoin
col_list2 = ["Zone","Code zone","Année","Produit","Code Produit",
             "Disponibilité intérieure","Aliments pour animaux","Semences","Pertes",
                 "Traitement","Nourriture","Autres utilisations (non alimentaire)"]
equilibre_prod = bilan_alim[col_list2]
# On les renomme 
equilibre_prod = equilibre_prod.rename(columns={"Code zone":"code_pays","Zone":"pays",
                                                "Année":"annee","Produit":"produit",
                                                "Code Produit":"code_produit",
                                                "Disponibilité intérieure":"dispo_int",
                                                "Aliments pour animaux":"alim_ani",
                                                "Semences":"semences","Pertes":"pertes",
                                                "Traitement":"transfo",
                                                "Nourriture":"nourriture",
                            "Autres utilisations (non alimentaire)":"autres_utilisations"})
equilibre_prod.head()

Élément,pays,code_pays,annee,produit,code_produit,dispo_int,alim_ani,semences,pertes,transfo,nourriture,autres_utilisations
0,Arménie,1,2013,Blé,2511,554.0,93.0,30.0,32.0,10.0,389.0,0.0
1,Arménie,1,2013,Orge,2513,198.0,137.0,14.0,15.0,7.0,0.0,26.0
2,Arménie,1,2013,Maïs,2514,102.0,96.0,0.0,7.0,NaN,0.0,NaN
3,Arménie,1,2013,Seigle,2515,1.0,1.0,0.0,0.0,NaN,0.0,NaN
4,Arménie,1,2013,Avoine,2516,6.0,4.0,0.0,0.0,NaN,1.0,NaN


*clé primaire (code_pays, code_produit, annee)*

In [10]:
# On procède à l'exportation SQL
equilibre_prod.to_csv("/Users/anissa/equilibre_prod.csv", index=False)

### Question n°18 - Export df sous_nutrition

In [11]:
# On sélectionne la tranche 2012-2014 pour effectuer nos calculs
sous_alimentation_2013 = sous_alimentation[sous_alimentation["Année"]\
                                                               .str.contains("2012-2014")]
# On sélectionne uniquement les données estimées par la FAO
sous_alimentation_2013 = sous_alimentation_2013.loc[
                        (sous_alimentation["Description du Symbole"]=="Estimation FAO")]
#  On supprime la donnée "<0.1"
sous_alimentation_2013 = sous_alimentation_2013[~sous_alimentation_2013['Valeur']\
                                                                        .isin(['<0.1'])]
# On procède à la conversion de la colonne en float
sous_alimentation_2013["Valeur"] = pd.to_numeric(sous_alimentation_2013["Valeur"], 
                                                                         errors='coerce')
# On renomme la colonne "Valeur" en "sous_alimentés"
sous_alimentation_2013 = sous_alimentation_2013.rename(columns={"Valeur":"sous_alimentés"})
# On supprime les données de la Chine (redondance)
sous_alim_2013 = sous_alimentation_2013.loc[sous_alimentation["Zone"]!="Chine"]
# On sélectionne les colonnes dont on a besoin
col_list = ["Zone","Code zone", "Année"]
sous_alim_2013 = sous_alim_2013[col_list]
# On ajoute la colonne pour l'avoir en millions
sous_alim_2013["nb_personnes"] = sous_alimentation_2013["sous_alimentés"]*1e6
# On les renomme
sous_alimentation_2013 = sous_alim_2013.rename(columns={"Zone":"pays",
                                                                "Code zone":"code_pays",
                                                                "Année":"annee"})
sous_alimentation_2013.head()

,pays,code_pays,annee,nb_personnes
0,Afghanistan,2,2012-2014,7900000.0
5,Afrique du Sud,202,2012-2014,2600000.0
10,Albanie,3,2012-2014,200000.0
15,Algérie,4,2012-2014,1700000.0
30,Angola,7,2012-2014,8100000.0


*clé primaire (code_pays, annee)*

In [12]:
# On procède à l'exportation SQL
sous_alimentation_2013.to_csv("/Users/anissa/sous_alim.csv", index=False)

## Résultats SQL obtenus

Nous allons afficher le résultat de chacune des requêtes grâce à une connexion entre notre base de données et Python.

### Question n°19 - Résultats des requêtes SQL

In [13]:
# On crée la connexion entre notre base de données et notre notebook
conn = sqlite3.connect("/Users/anissa/P03_BDD.db")
cur = conn.cursor()

*Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant*

In [14]:
req_1 = pd.read_sql_query(
    """SELECT pays, sum(dispo_prot/1000) AS dispo_prot_habitant 
        FROM dispo_alim 
        GROUP BY pays 
        ORDER BY dispo_prot_habitant 
        DESC LIMIT 10""", conn)
req_1

,pays,dispo_prot_habitant
0,Islande,0.13306
1,Chine - RAS de Hong-Kong,0.12907
2,Israël,0.12800
3,Lituanie,0.12436
4,Maldives,0.12232
5,Finlande,0.11756
6,Luxembourg,0.11364
7,Monténégro,0.11190
8,Pays-Bas,0.11146
9,Albanie,0.11137


*Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de kcal par habitant*

In [15]:
req_2 = pd.read_sql_query(
    """SELECT pays, sum(dispo_alim_kcal_p_j) AS dispo_kcal_habitant 
        FROM dispo_alim 
        GROUP BY pays 
        ORDER BY dispo_kcal_habitant 
        DESC LIMIT 10""", conn)
req_2

,pays,dispo_kcal_habitant
0,Autriche,3770.0
1,Belgique,3737.0
2,Turquie,3708.0
3,États-Unis d'Amérique,3682.0
4,Israël,3610.0
5,Irlande,3602.0
6,Italie,3578.0
7,Luxembourg,3540.0
8,Égypte,3518.0
9,Allemagne,3503.0


*Pour l'année 2013, les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant*

In [21]:
req_3 = pd.read_sql_query(
    """SELECT pays, sum(dispo_prot/1000) AS dispo_prot_habitant_kg 
        FROM dispo_alim 
        GROUP BY pays 
        ORDER BY dispo_prot_habitant_kg 
        ASC LIMIT 10""", conn)
req_3

,pays,dispo_prot_habitant_kg
0,Libéria,0.03766
1,Guinée-Bissau,0.04405
2,Mozambique,0.04568
3,République centrafricaine,0.04604
4,Madagascar,0.04669
5,Haïti,0.04770
6,Zimbabwe,0.04832
7,Congo,0.05141
8,Ouganda,0.05264
9,Sao Tomé-et-Principe,0.05310


*La quantité totale (en kg) de produits perdus par pays en 2013*

In [17]:
req_4 = pd.read_sql_query(
    """SELECT pays, annee, sum(pertes)*1000000 
        FROM equilibre_prod 
        GROUP BY pays""", conn)
req_4

,pays,annee,sum(pertes)*1000000
0,Afghanistan,2013,1.135000e+09
1,Afrique du Sud,2013,2.193000e+09
2,Albanie,2013,2.760000e+08
3,Algérie,2013,3.753000e+09
4,Allemagne,2013,3.781000e+09
...,...,...,...
169,Émirats arabes unis,2013,7.050000e+08
170,Équateur,2013,7.070000e+08
171,États-Unis d'Amérique,2013,7.162000e+09
172,Éthiopie,2013,2.256000e+09


*Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte*

In [18]:
req_5 = pd.read_sql_query(
    """SELECT S.pays,(S.nb_personnes /P.population)*100 as prop_sous_alim   
        FROM sous_nutrition S, population P
        WHERE P. code_pays = S. code_pays
        ORDER BY prop_sous_alim  DESC
        LIMIT 10""", conn)
req_5

,pays,prop_sous_alim
0,Haïti,50.402249
1,Zambie,48.146365
2,Zimbabwe,46.643110
3,République centrafricaine,43.327556
4,République populaire démocratique de Corée,42.578831
5,Congo,40.467626
6,Tchad,38.206628
7,Angola,37.723547
8,Libéria,37.261295
9,Madagascar,35.768811


*Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé*

In [19]:
req_6 = pd.read_sql_query(
   """SELECT produit, avg(autres_utilisations /dispo_int) as ratio_others_dispo FROM equilibre_prod
        GROUP BY produit
        ORDER BY  ratio_others_dispo DESC
        LIMIT 10""",conn)
req_6

,produit,ratio_others_dispo
0,"Alcool, non Comestible",0.982456
1,Plantes Aquatiques,0.920661
2,Huile de Palmistes,0.773645
3,Piments,0.739130
4,Huile de Palme,0.652520
5,Huile de Colza&Moutarde,0.616865
6,Palmistes,0.575625
7,Huile de Coco,0.570894
8,Huil Plantes Oleif Autr,0.547179
9,Huile de Son de Riz,0.502955


>> Quelles sont les autres utilisations possibles pour ces 10 produits ?
Alcool : usage médical, cosmétique et combustible -
Plantes aquatiques : aquariophilie -
Huile de palmistes, huile de palme, huile de colza, palmistes, huile de coco, huile de plantes oleif, huile de son de riz : bio-carburants et cosmétique -
Piments : usage médicinal